d ##### Usecase:Get crimes data by type
###### Solutions are given for following problem statements
###### Problem statement-1:Get crimes data per day
###### Problem statement-2:Get crimes data per month
###### Problem statement-3:Get crimes data per year
###### Problem statement-4:Get TOP3 crime types based on number of incidents in RESIDENCE
###### Problem statement-5:Get yearly crimes data from 2001 to 2020 with count
###### Problem statement-6:Get yearly crime data of location BANK
###### Problem statement-7:Get yearly crimes data  of DOMESTIC incidents for all years with location 
###### Problem statement-8:Get yearly crimes data of crimetype BURGLARY happened in residence, park and apartments
###### Problem statement-9:Get yearly crimes data of THEFT incidents between 2015 and 2017
###### Problem statement-10:Get yearly crimes data of NARCOTICS incidents happened in AIRPORTS

In [2]:
from pyspark.sql import SQLContext, Row
from pyspark.sql import HiveContext
from pyspark.sql.functions import round,sum,avg,count,substring
hiveContext = HiveContext(sc)
sqlContext.sql("set spark.sql.shuffle.partitions=10")

Out[6]: DataFrame[key: string, value: string]

In [3]:
hiveContext.sql("create database crime_db")
hiveContext.sql("use crime_db")

In [4]:
hiveContext.sql("create table crime('ID','CaseNumber','Date','Block','IUCR','PrimaryType','Description',locationDescription','Arrest','Domestic','Beat','District','Ward', \                                                                       'CommunityArea','FBICode','XCoordinate','YCoordinate','Year','UpdatedOn','Latitude','Longitude','Location') \
                                     row format delimited fields terminated by ','")

In [5]:
hiveContext.sql("load data local inpath 'dbfs:/FileStore/tables/crimes_data.csv' into table crime")

In [6]:
hiveContext.sql("show tables").show()

+--------+---------+-----------+
database|tableName|isTemporary|
+--------+---------+-----------+
crime_db| crime| false|
crime_db| crimes| false|
+--------+---------+-----------+

In [7]:
hiveContext.sql("select * from crime").show(5)

+--------+----------+---------------+--------------------+----+-------------------+--------------------+-------------------+------+--------+----+--------+----+-------------+-------+-----------+-----------+----+---------------+--------+---------+--------+
 ID|CaseNumber| Date| Block|IUCR| PrimaryType| Description|LocationDescription|Arrest|Domestic|Beat|District|Ward|CommunityArea|FBICode|XCoordinate|YCoordinate|Year| UpdatedOn|Latitude|Longitude|Location|
+--------+----------+---------------+--------------------+----+-------------------+--------------------+-------------------+------+--------+----+--------+----+-------------+-------+-----------+-----------+----+---------------+--------+---------+--------+
11034701| JA366925| 1/1/2001 11:00| 016XX E 86TH PL|1153| DECEPTIVE PRACTICE|FINANCIAL IDENTIT...| RESIDENCE| false| false| 412| 4| 8| 45| 11| null| null|2001| 8/5/2017 15:50| null| null| null|
11227287| JB147188| 10/8/2017 3:00| 092XX S RACINE AVE| 281|CRIM SEXUAL ASSAULT| NON-AGGRAVATED| RESIDENCE| false| false|2222| 22| 21| 73| 2| null| null|2017|2/11/2018 15:57| null| null| null|
11227583| JB147595|3/28/2017 14:00| 026XX W 79TH ST| 620| BURGLARY| UNLAWFUL ENTRY| OTHER| false| false| 835| 8| 18| 70| 5| null| null|2017|2/11/2018 15:57| null| null| null|
11227293| JB147230| 9/9/2017 20:17|060XX S EBERHART AVE| 810| THEFT| OVER $500| RESIDENCE| false| false| 313| 3| 20| 42| 6| null| null|2017|2/11/2018 15:57| null| null| null|
11227634| JB147599|8/26/2017 10:00| 001XX W RANDOLPH ST| 281|CRIM SEXUAL ASSAULT| NON-AGGRAVATED| HOTEL/MOTEL| false| false| 122| 1| 42| 32| 2| null| null|2017|2/11/2018 15:57| null| null| null|
+--------+----------+---------------+--------------------+----+-------------------+--------------------+-------------------+------+--------+----+--------+----+-------------+-------+-----------+-----------+----+---------------+--------+---------+--------+
only showing top 5 rows

In [8]:
hiveContext.sql("select * from crime").printSchema()

root
-- ID: integer (nullable = true)
-- CaseNumber: string (nullable = true)
-- Date: string (nullable = true)
-- Block: string (nullable = true)
-- IUCR: string (nullable = true)
-- PrimaryType: string (nullable = true)
-- Description: string (nullable = true)
-- LocationDescription: string (nullable = true)
-- Arrest: boolean (nullable = true)
-- Domestic: boolean (nullable = true)
-- Beat: integer (nullable = true)
-- District: integer (nullable = true)
-- Ward: integer (nullable = true)
-- CommunityArea: integer (nullable = true)
-- FBICode: string (nullable = true)
-- XCoordinate: string (nullable = true)
-- YCoordinate: string (nullable = true)
-- Year: integer (nullable = true)
-- UpdatedOn: string (nullable = true)
-- Latitude: string (nullable = true)
-- Longitude: string (nullable = true)
-- Location: string (nullable = true)

###### Problem statement-1:Get crimes data per day

In [10]:
CrimesDataPerday=hiveContext.sql('''select to_date(from_unixtime(UNIX_TIMESTAMP(crmtbl.Date,'MM/dd/yyyy'))) as crimedate,
                                      crmtbl.PrimaryType as crime_Type, 
                                      count(crmtbl.PrimaryType) as NumberofCrimePerTypes
                                      from crime crmtbl 
                                      group by to_date(from_unixtime(UNIX_TIMESTAMP(crmtbl.Date,'MM/dd/yyyy'))),crmtbl.PrimaryType 
                                      order by to_date(from_unixtime(UNIX_TIMESTAMP(crmtbl.Date,'MM/dd/yyyy'))),crmtbl.PrimaryType''')
CrimesDataPerDay.coalesce(1).write.csv("/FileStore/tables/CrimesDataPerDay", compression="none", header ='true')
CrimesDataPerday.show(10,False)

+----------+--------------------------+---------------------+
crimedate |crime_Type |NumberofCrimePerTypes|
+----------+--------------------------+---------------------+
2001-01-01|DECEPTIVE PRACTICE |1 |
2001-01-01|OFFENSE INVOLVING CHILDREN|1 |
2001-01-03|MOTOR VEHICLE THEFT |1 |
2001-01-07|ASSAULT |1 |
2001-01-09|CRIMINAL DAMAGE |1 |
2001-01-10|THEFT |1 |
2001-01-13|ARSON |1 |
2001-01-13|NARCOTICS |1 |
2001-01-14|ASSAULT |1 |
2001-01-18|THEFT |1 |
+----------+--------------------------+---------------------+
only showing top 10 rows

###### Problem statement-2:Get crimes data per month

In [12]:
crimesDataPerMonth=hiveContext.sql('''select substring(to_date(from_unixtime(UNIX_TIMESTAMP(crmtbl.Date,'MM/dd/yyyy'))),0,7) as crimemonth,           
                                                  crmtbl.PrimaryType as crime_Type, 
                                                  count(crmtbl.PrimaryType) as NumberofCrimePerTypes
                                                  from crime crmtbl 
                                                  group by crimemonth,crmtbl.PrimaryType 
                                                  order by crimemonth,crmtbl.PrimaryType''')
crimesDataPerMonth.coalesce(1).write.csv("/FileStore/tables/departmentsSalesDataPerMonth", compression="none", header ='true')
crimesDataPerMonth.show(20,False)

+----------+--------------------------+---------------------+
crimemonth|crime_Type |NumberofCrimePerTypes|
+----------+--------------------------+---------------------+
2001-01 |ARSON |1 |
2001-01 |ASSAULT |2 |
2001-01 |BURGLARY |1 |
2001-01 |CRIMINAL DAMAGE |1 |
2001-01 |DECEPTIVE PRACTICE |2 |
2001-01 |MOTOR VEHICLE THEFT |3 |
2001-01 |NARCOTICS |4 |
2001-01 |OFFENSE INVOLVING CHILDREN|1 |
2001-01 |THEFT |4 |
2001-01 |WEAPONS VIOLATION |1 |
2001-02 |ASSAULT |1 |
2001-02 |BATTERY |1 |
2001-02 |CRIMINAL DAMAGE |2 |
2001-02 |CRIMINAL TRESPASS |1 |
2001-02 |NARCOTICS |3 |
2001-02 |ROBBERY |4 |
2001-02 |THEFT |6 |
2001-03 |ASSAULT |1 |
2001-03 |BATTERY |3 |
2001-03 |CRIM SEXUAL ASSAULT |2 |
+----------+--------------------------+---------------------+
only showing top 20 rows

###### Problem statement-3:Get crimes data per year

In [14]:
crimesDataPerYear=hiveContext.sql('''select substring(to_date(from_unixtime(UNIX_TIMESTAMP(crmtbl.Date,'MM/dd/yyyy'))),0,4) as crimeyear,           
                                                  crmtbl.PrimaryType as crime_Type, 
                                                  count(crmtbl.PrimaryType) as NumberofCrimePerTypes
                                                  from crime crmtbl 
                                                  group by crimeyear,crmtbl.PrimaryType 
                                                  order by crimeyear,crmtbl.PrimaryType''')
crimesDataPerYear.coalesce(1).write.csv("/FileStore/tables/crimesDataPerYear", compression="none", header ='true')
crimesDataPerYear.show(20,False)

+---------+--------------------------+---------------------+
crimeyear|crime_Type |NumberofCrimePerTypes|
+---------+--------------------------+---------------------+
2001 |ARSON |1 |
2001 |ASSAULT |19 |
2001 |BATTERY |26 |
2001 |BURGLARY |5 |
2001 |CRIM SEXUAL ASSAULT |2 |
2001 |CRIMINAL DAMAGE |22 |
2001 |CRIMINAL SEXUAL ASSAULT |1 |
2001 |CRIMINAL TRESPASS |6 |
2001 |DECEPTIVE PRACTICE |9 |
2001 |LIQUOR LAW VIOLATION |2 |
2001 |MOTOR VEHICLE THEFT |24 |
2001 |NARCOTICS |37 |
2001 |OFFENSE INVOLVING CHILDREN|1 |
2001 |OTHER OFFENSE |8 |
2001 |PROSTITUTION |3 |
2001 |PUBLIC PEACE VIOLATION |3 |
2001 |ROBBERY |18 |
2001 |SEX OFFENSE |1 |
2001 |THEFT |70 |
2001 |WEAPONS VIOLATION |6 |
+---------+--------------------------+---------------------+
only showing top 20 rows

###### Problem statement-4:Get TOP3 crime types based on number of incidents in RESIDENCE

In [16]:
top3crimetypes=sqlContext.sql('''select cmtl.PrimaryType as CrimeType, count(*) as NumberofIncidents from crime cmtl
                                  where cmtl.LocationDescription="RESIDENCE" 
                                  group by CrimeType 
                                  order by NumberofIncidents desc limit 3''')
top3crimetypes.coalesce(1).write.csv("/FileStore/tables/top3crimetypesr", compression="none", header ='true')
top3crimetypes.show(5,False)

+--------------------------+-----------------+
CrimeType |NumberofIncidents|
+--------------------------+-----------------+
DECEPTIVE PRACTICE |456 |
THEFT |201 |
OFFENSE INVOLVING CHILDREN|152 |
+--------------------------+-----------------+

###### Problem statement-5:Get yearly crimes data from 2001 to 2020 with count

In [18]:
NoofcrimesPerYear=hiveContext.sql('''select substring(to_date(from_unixtime(UNIX_TIMESTAMP(crmtbl.Date,'MM/dd/yyyy'))),0,4) as crimeyear,           
                                                  count(crmtbl.PrimaryType) as NumberofCrimesPeryear
                                                  from crime crmtbl 
                                                  group by crimeyear 
                                                  order by crimeyear''')
NoofcrimesPerYear.coalesce(1).write.csv("/FileStore/tables/NoofcrimesPerYear", compression="none", header ='true')
NoofcrimesPerYear.show(25,False)

+---------+---------------------+
crimeyear|NumberofCrimesPeryear|
+---------+---------------------+
2001 |264 |
2002 |1364 |
2003 |379 |
2004 |207 |
2005 |333 |
2006 |94 |
2007 |44 |
2008 |108 |
2009 |49 |
2010 |55 |
2011 |55 |
2012 |90 |
2013 |122 |
2014 |241 |
2015 |359 |
2016 |182 |
2017 |673 |
2018 |335 |
2019 |45 |
+---------+---------------------+

###### Problem statement-6:Get yearly crime data of location BANK

In [20]:
crimesDataofbanks = hiveContext.sql('''select substring(to_date(from_unixtime(UNIX_TIMESTAMP(crmtbl.Date,'MM/dd/yyyy'))),0,4) as crimeyear,           
                                                  crmtbl.LocationDescription,crmtbl.PrimaryType as crimetype,count(crmtbl.PrimaryType) as NumberofCrimesPeryear
                                                  from crime crmtbl 
                                                  where crmtbl.LocationDescription = 'BANK'
                                                  group by crimeyear,crmtbl.LocationDescription,crmtbl.PrimaryType
                                                  order by crimeyear,crmtbl.LocationDescription,crmtbl.PrimaryType''')
crimesDataofbanks.coalesce(1).write.csv("/FileStore/tables/crimesDataofbanks", compression="none", header ='true')
crimesDataofbanks.show(30,False)

+---------+-------------------+------------------+---------------------+
crimeyear|LocationDescription|crimetype |NumberofCrimesPeryear|
+---------+-------------------+------------------+---------------------+
2001 |BANK |DECEPTIVE PRACTICE|1 |
2001 |BANK |THEFT |1 |
2002 |BANK |DECEPTIVE PRACTICE|5 |
2002 |BANK |ROBBERY |1 |
2002 |BANK |THEFT |4 |
2003 |BANK |DECEPTIVE PRACTICE|3 |
2003 |BANK |NARCOTICS |1 |
2005 |BANK |DECEPTIVE PRACTICE|4 |
2006 |BANK |DECEPTIVE PRACTICE|2 |
2009 |BANK |DECEPTIVE PRACTICE|1 |
2011 |BANK |DECEPTIVE PRACTICE|2 |
2012 |BANK |DECEPTIVE PRACTICE|1 |
2013 |BANK |DECEPTIVE PRACTICE|4 |
2014 |BANK |DECEPTIVE PRACTICE|1 |
2015 |BANK |DECEPTIVE PRACTICE|6 |
2016 |BANK |DECEPTIVE PRACTICE|2 |
2017 |BANK |DECEPTIVE PRACTICE|6 |
2017 |BANK |THEFT |1 |
2018 |BANK |BATTERY |1 |
2018 |BANK |DECEPTIVE PRACTICE|5 |
+---------+-------------------+------------------+---------------------+

###### Problem statement-7:Get yearly crimes data  of DOMESTIC incidents for all years with location

In [22]:
crimesDataofdomestic = hiveContext.sql('''select substring(to_date(from_unixtime(UNIX_TIMESTAMP(crmtbl.Date,'MM/dd/yyyy'))),0,4) as crimeyear,           
                                                  crmtbl.LocationDescription,count(crmtbl.PrimaryType) as NumberofCrimes
                                                  from crime crmtbl 
                                                  where Domestic = 'TRUE'
                                                  group by crimeyear,crmtbl.LocationDescription 
                                                  order by crimeyear,crmtbl.LocationDescription''')
crimesDataofdomestic.coalesce(1).write.csv("/FileStore/tables/crimesDataofdomestic", compression="none", header ='true')Domestic
crimesDataofdomestic.show(20,False)

+---------+------------------------------+--------------+
crimeyear|LocationDescription |NumberofCrimes|
+---------+------------------------------+--------------+
2001 |APARTMENT |2 |
2001 |PARKING LOT/GARAGE(NON.RESID.)|1 |
2001 |RESIDENCE |3 |
2001 |STREET |3 |
2002 |ALLEY |2 |
2002 |APARTMENT |7 |
2002 |CHA APARTMENT |1 |
2002 |COMMERCIAL / BUSINESS OFFICE |1 |
2002 |CURRENCY EXCHANGE |1 |
2002 |GAS STATION |1 |
2002 |HIGHWAY/EXPRESSWAY |2 |
2002 |HOSPITAL BUILDING/GROUNDS |1 |
2002 |PARK PROPERTY |1 |
2002 |PARKING LOT/GARAGE(NON.RESID.)|1 |
2002 |RESIDENCE |19 |
2002 |SIDEWALK |7 |
2002 |STREET |37 |
2002 |VEHICLE NON-COMMERCIAL |3 |
2003 |APARTMENT |10 |
2003 |BANK |1 |
+---------+------------------------------+--------------+
only showing top 20 rows

###### Problem statement-8:Get yearly crimes data of crimetype BURGLARY happened in residence, park and apartments

In [24]:
crimesDataofburglary = hiveContext.sql('''select substring(to_date(from_unixtime(UNIX_TIMESTAMP(crmtbl.Date,'MM/dd/yyyy'))),0,4) as crimeyear,           
                                                  crmtbl.PrimaryType as crimetype,crmtbl.LocationDescription,count(crmtbl.PrimaryType) as NumberofCrimes
                                                  from crime crmtbl 
                                                  where crmtbl.PrimaryType LIKE "%BURGLARY%" and 
                                                  (crmtbl.LocationDescription LIKE "%RESIDENCE%"or 
                                                    crmtbl.LocationDescription LIKE "PARK" or crmtbl.LocationDescription LIKE "APARTMENT")
                                                  group by crimeyear,crimetype,crmtbl.LocationDescription 
                                                  order by crimeyear,crimetype,crmtbl.LocationDescription''')
crimesDataofburglary.coalesce(1).write.csv("/FileStore/tables/crimesDataofburglary", compression="none", header ='true')Domestic
crimesDataofburglary.show(30,False)

+---------+---------+-------------------+--------------+
crimeyear|crimetype|LocationDescription|NumberofCrimes|
+---------+---------+-------------------+--------------+
2001 |BURGLARY |RESIDENCE |2 |
2002 |BURGLARY |APARTMENT |4 |
2002 |BURGLARY |RESIDENCE |5 |
2002 |BURGLARY |RESIDENCE-GARAGE |2 |
2003 |BURGLARY |APARTMENT |2 |
2003 |BURGLARY |RESIDENCE |4 |
2003 |BURGLARY |RESIDENCE-GARAGE |2 |
2004 |BURGLARY |APARTMENT |1 |
2005 |BURGLARY |RESIDENCE-GARAGE |1 |
2006 |BURGLARY |APARTMENT |1 |
2007 |BURGLARY |APARTMENT |1 |
2008 |BURGLARY |RESIDENCE-GARAGE |1 |
2014 |BURGLARY |RESIDENCE |1 |
2015 |BURGLARY |RESIDENCE |1 |
2017 |BURGLARY |APARTMENT |4 |
2017 |BURGLARY |RESIDENCE |7 |
2017 |BURGLARY |RESIDENCE-GARAGE |2 |
2018 |BURGLARY |APARTMENT |2 |
+---------+---------+-------------------+--------------+

###### Problem statement-9:Get yearly crimes data of THEFT incidents between 2015 and 2017

In [26]:
crimesDataoftheft = hiveContext.sql('''select substring(to_date(from_unixtime(UNIX_TIMESTAMP(crmtbl.Date,'MM/dd/yyyy'))),0,4) as crimeyear,           
                                                  crmtbl.PrimaryType as crimetype,crmtbl.LocationDescription,count(crmtbl.PrimaryType) as NumberofCrimes
                                                  from crime crmtbl 
                                                  where crmtbl.PrimaryType LIKE "%THEFT%" and (crmtbl.Year>=2015 AND crmtbl.Year<=2017) 
                                                  group by crimeyear,crmtbl.PrimaryType,crmtbl.LocationDescription 
                                                  having count(crmtbl.PrimaryType)>=4
                                                  order by crimeyear''')
crimesDataoftheft.coalesce(1).write.csv("/FileStore/tables/crimesDataoftheft", compression="none", header ='true')
crimesDataoftheft.show(50,False)

+---------+---------+-----------------------+--------------+
crimeyear|crimetype|LocationDescription |NumberofCrimes|
+---------+---------+-----------------------+--------------+
2015 |THEFT |OTHER |4 |
2015 |THEFT |STREET |6 |
2015 |THEFT |RESIDENCE |5 |
2015 |THEFT |APARTMENT |5 |
2016 |THEFT |STREET |9 |
2016 |THEFT |PARK PROPERTY |4 |
2016 |THEFT |RESIDENCE PORCH/HALLWAY|5 |
2017 |THEFT |ALLEY |6 |
2017 |THEFT |RESIDENCE |23 |
2017 |THEFT |OTHER |13 |
2017 |THEFT |STREET |19 |
2017 |THEFT |RESTAURANT |6 |
2017 |THEFT |RESIDENCE PORCH/HALLWAY|7 |
2017 |THEFT |VEHICLE NON-COMMERCIAL |4 |
2017 |THEFT |SIDEWALK |5 |
2017 |THEFT |PARK PROPERTY |11 |
2017 |THEFT |BAR OR TAVERN |5 |
2017 |THEFT |APARTMENT |18 |
+---------+---------+-----------------------+--------------+

###### Problem statement-10:Get monthly crimes data of NARCOTICS incidents happened in AIRPORTS

In [28]:
crimesDataofnarcotics = hiveContext.sql('''select substring(to_date(from_unixtime(UNIX_TIMESTAMP(crmtbl.Date,'MM/dd/yyyy'))),0,7) as crimemonth,           
                                                  crmtbl.PrimaryType as crimetype,crmtbl.LocationDescription,count(crmtbl.PrimaryType) as NumberofCrimes
                                                  from crime crmtbl 
                                                  where crmtbl.PrimaryType = "NARCOTICS" and crmtbl.LocationDescription LIKE "%AIRPORT%"
                                                  group by crimemonth,crmtbl.PrimaryType,crmtbl.LocationDescription
                                                  order by crimemonth''')
crimesDataofnarcotics.coalesce(1).write.csv("/FileStore/tables/crimesDataofnarcotics", compression="none", header ='true')
crimesDataofnarcotics.show(20,False)

+----------+---------+-------------------+--------------+
crimemonth|crimetype|LocationDescription|NumberofCrimes|
+----------+---------+-------------------+--------------+
2002-03 |NARCOTICS|AIRPORT/AIRCRAFT |1 |
2014-12 |NARCOTICS|AIRPORT/AIRCRAFT |4 |
2015-07 |NARCOTICS|AIRPORT/AIRCRAFT |1 |
2015-10 |NARCOTICS|AIRPORT/AIRCRAFT |2 |
2017-09 |NARCOTICS|AIRPORT/AIRCRAFT |1 |
2017-10 |NARCOTICS|AIRPORT/AIRCRAFT |1 |
2017-11 |NARCOTICS|AIRPORT/AIRCRAFT |1 |
+----------+---------+-------------------+--------------+